In [48]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [49]:
item_categories = pd.read_csv('../data/item_categories.csv')
item_categories.head()

,item_category_name,item_category_id
0,PC - Гарнитуры/Наушники,0
1,Аксессуары - PS2,1
2,Аксессуары - PS3,2
3,Аксессуары - PS4,3
4,Аксессуары - PSP,4


In [50]:
shops = pd.read_csv('../data/shops.csv')
shops.head()

,shop_name,shop_id
0,"!Якутск Орджоникидзе, 56 фран",0
1,"!Якутск ТЦ ""Центральный"" фран",1
2,"Адыгея ТЦ ""Мега""",2
3,"Балашиха ТРК ""Октябрь-Киномир""",3
4,"Волжский ТЦ ""Волга Молл""",4


In [51]:
items = pd.read_csv('../data/items.csv')
items.head()

,item_name,item_id,item_category_id
0,! ВО ВЛАСТИ НАВАЖДЕНИЯ (ПЛАСТ.) D,0,40
1,!ABBYY FineReader 12 Professional Edition Full...,1,76
2,***В ЛУЧАХ СЛАВЫ (UNV) D,2,40
3,***ГОЛУБАЯ ВОЛНА (Univ) D,3,40
4,***КОРОБКА (СТЕКЛО) D,4,40


In [52]:
sales_train = pd.read_csv('../data/sales_train.csv')
sales_train.head()

,date,date_block_num,shop_id,item_id,item_price,item_cnt_day
0,02.01.2013,0,59,22154,999.00,1.0
1,03.01.2013,0,25,2552,899.00,1.0
2,05.01.2013,0,25,2552,899.00,-1.0
3,06.01.2013,0,25,2554,1709.05,1.0
4,15.01.2013,0,25,2555,1099.00,1.0


In [53]:
shops['city'] = shops['shop_name'].str.extract(r'(\w+\.*\w*)')
shops.head()

,shop_name,shop_id,city
0,"!Якутск Орджоникидзе, 56 фран",0,Якутск
1,"!Якутск ТЦ ""Центральный"" фран",1,Якутск
2,"Адыгея ТЦ ""Мега""",2,Адыгея
3,"Балашиха ТРК ""Октябрь-Киномир""",3,Балашиха
4,"Волжский ТЦ ""Волга Молл""",4,Волжский


In [54]:
items_with_categories = items.merge(item_categories, how='outer', on='item_category_id')
items_with_categories.head()

,item_name,item_id,item_category_id,item_category_name
0,! ВО ВЛАСТИ НАВАЖДЕНИЯ (ПЛАСТ.) D,0,40,Кино - DVD
1,***В ЛУЧАХ СЛАВЫ (UNV) D,2,40,Кино - DVD
2,***ГОЛУБАЯ ВОЛНА (Univ) D,3,40,Кино - DVD
3,***КОРОБКА (СТЕКЛО) D,4,40,Кино - DVD
4,***НОВЫЕ АМЕРИКАНСКИЕ ГРАФФИТИ (UNI) ...,5,40,Кино - DVD


In [62]:
data = sales_train.merge(shops.iloc[:, 1:], on='shop_id')
data = data.merge(items_with_categories.drop(columns='item_name'), on='item_id')
data.head()

,date,date_block_num,shop_id,item_id,item_price,item_cnt_day,city,item_category_id,item_category_name
0,02.01.2013,0,59,22154,999.0,1.0,Ярославль,37,Кино - Blu-Ray
1,02.01.2013,0,25,22154,999.0,1.0,Москва,37,Кино - Blu-Ray
2,03.01.2013,0,25,22154,999.0,1.0,Москва,37,Кино - Blu-Ray
3,20.01.2013,0,25,22154,999.0,1.0,Москва,37,Кино - Blu-Ray
4,23.01.2013,0,25,22154,999.0,1.0,Москва,37,Кино - Blu-Ray


In [63]:
data[data['item_cnt_day'] < 0].shape

(7356, 9)

In [64]:
data[data['item_price'] < 0].shape

(1, 9)

* There is negatives in price and num of sold items fields

In [65]:
data.loc[data.item_cnt_day < 1, "item_cnt_day"] = 0
data = data[data.item_price > 0].reset_index(drop = True)

### Check missing values and duplicates

In [66]:
data.isnull().sum()

date                  0
date_block_num        0
shop_id               0
item_id               0
item_price            0
item_cnt_day          0
city                  0
item_category_id      0
item_category_name    0
dtype: int64

Omg... no missing values

In [67]:
data.duplicated().sum()

6

In [68]:
data = data.drop_duplicates().reset_index(drop=True)
data.head()

,date,date_block_num,shop_id,item_id,item_price,item_cnt_day,city,item_category_id,item_category_name
0,02.01.2013,0,59,22154,999.0,1.0,Ярославль,37,Кино - Blu-Ray
1,02.01.2013,0,25,22154,999.0,1.0,Москва,37,Кино - Blu-Ray
2,03.01.2013,0,25,22154,999.0,1.0,Москва,37,Кино - Blu-Ray
3,20.01.2013,0,25,22154,999.0,1.0,Москва,37,Кино - Blu-Ray
4,23.01.2013,0,25,22154,999.0,1.0,Москва,37,Кино - Blu-Ray


In [69]:
data.describe()

,date_block_num,shop_id,item_id,item_price,item_cnt_day,item_category_id
count,2.935842e+06,2.935842e+06,2.935842e+06,2.935842e+06,2.935842e+06,2.935842e+06
mean,1.456992e+01,3.300171e+01,1.019723e+04,8.908538e+02,1.245210e+00,4.000141e+01
std,9.422992e+00,1.622698e+01,6.324292e+03,1.729802e+03,2.617049e+00,1.710076e+01
min,0.000000e+00,0.000000e+00,0.000000e+00,7.000000e-02,0.000000e+00,0.000000e+00
25%,7.000000e+00,2.200000e+01,4.476000e+03,2.490000e+02,1.000000e+00,2.800000e+01
50%,1.400000e+01,3.100000e+01,9.343000e+03,3.990000e+02,1.000000e+00,4.000000e+01
75%,2.300000e+01,4.700000e+01,1.568400e+04,9.990000e+02,1.000000e+00,5.500000e+01
max,3.300000e+01,5.900000e+01,2.216900e+04,3.079800e+05,2.169000e+03,8.300000e+01


In [72]:
print(items.item_name.nunique())
items.shape

22170


(22170, 3)